[Reference](https://medium.com/pipeline-a-data-engineering-resource/a-simple-python-script-that-allows-me-to-save-more-than-20k-a-year-pt-ii-a88ae433276e)

In [2]:
import pandas as pd
import numpy as np
import logging
import os
from datetime import date
from google.cloud import bigquery
import config as cfg

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=cfg.creds

logging.basicConfig(format='%(asctime)s %(message)s',level=logging.INFO)

bq_client = bigquery.Client()

current_dt = date.today()

bills = {
        
        "rent": 1300,
        "washer_dryer": 40,
        "loans": 150,
        "car_insurance_zach": 60,
        "car_insurance_wife": 65,
        "phones": 120,
        "internet": 100,
        "utilities": 28,
        "electric": 125,
        "cred_card": 300
        
    }

def yearly_after_tax(gross: int, tax: int):
    
    gross = gross
    tax = tax
    
    money_in = gross - tax
    
    print(f"Annual money in: ${money_in:,.2f}.")
    
    col_dict = bills
    
    cost_of_living_total = sum(col_dict.values())
    
    cost_of_living_total = cost_of_living_total * 12
    
    after_col = money_in - cost_of_living_total
    
    print(f"After cost of living: ${after_col:,.2f}.")
    
    savings = after_col * 0.3
    
    print(f"Deposit ${savings:,.2f} into savings account.")
    
    discretionary = after_col - savings
    
    return print(f"Discretionary funds remaining for the year: ${round(discretionary):,.2f}")
    
def monthly(bills: float, expense: float, paycheck_1: int, paycheck_2: int, paycheck_3: int):
    
    zach_paycheck_1 = paycheck_1
    zach_paycheck_2 = paycheck_2
    
    wife_paycheck = paycheck_3
    
    monthly_total = zach_paycheck_1 + zach_paycheck_2 + wife_paycheck
    
    bills = bills
    
    bills_total = sum(bills.values())
    
    monthly_remaining = monthly_total - bills_total
    
    savings_cut = monthly_remaining * 0.3
    
    expense_total = expense
    
    discretionary = monthly_remaining - expense_total
    
    return print(f"Money remaining for this month is: ${round(discretionary):,.2f}")

def main():
    
    logging.info(f"Calculating yearly and monthly discretionary income for {current_dt}")
    
    query = """ 

    WITH transaction AS (
    SELECT * FROM (
    SELECT description, amount, ROW_NUMBER() OVER(PARTITION BY description) AS row_num FROM `**************.mint.transactions`
    WHERE date BETWEEN "2022-06-01" AND "2022-06-30"
    AND description NOT LIKE "%Online Transfer To Sav%" AND description NOT LIKE "%Online Transfer From Sav%" AND description NOT LIKE "Online Transfer To Chk%" AND description NOT LIKE "Venmo%"
    )
    WHERE row_num = 1
    )

    SELECT * EXCEPT(row_num) FROM transaction
    WHERE amount < 0

    """

    df = bq_client.query(query).to_dataframe()

    df["amount"] = df["amount"].abs()
    total = df["amount"].sum()
    
    yearly_after_tax(75000, 15000)
    
    monthly(bills, total, 2000, 2000, 1000)
    
if __name__ == "__main__":
    main()